In [9]:
from google.colab import drive


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install paddleocr -q
!python3 -m pip install paddlepaddle-gpu -f https://paddlepaddle.org.cn/whl/stable.html -q

In [ ]:
from paddleocr import PaddleOCR, draw_ocr
import paddle
import os

gpu_available  = paddle.device.is_compiled_with_cuda()
print("GPU available:", gpu_available)

# ocr = PaddleOCR(lang="korean",use_gpu=True) # The model file will be downloaded automatically when executed for the first time
ocr = PaddleOCR(lang="korean",use_gpu=True) # The model file will be downloaded automatically when executed for the first time
result = []
import time
start = time.time()

img_path = '/content/drive/MyDrive/bus2.jpg'
print(img_path)
result.extend(ocr.ocr(img_path,cls=False))
end = time.time()
elapsed_time = end - start
print(f"작업에 걸린 시간: {elapsed_time}초")

# Recognition and detection can be performed separately through parameter control
# result = ocr.ocr(img_path, det=False)  Only perform recognition
# result = ocr.ocr(img_path, rec=False)  Only perform detection
# Print detection frame and recognition result
for line in result:
    print(line) # 없으면 None



In [ ]:
start = time.time()
print(img_path)
result = ocr.ocr(img_path,cls=False)
end = time.time()
elapsed_time = end - start
print(f"작업에 걸린 시간: {elapsed_time}초")

len(result)


In [ ]:
result

In [ ]:
!paddleocr --image_dir /content/drive/MyDrive/iSEE/data/Training/원천데이터/test5.png --lang=korean

In [ ]:
#######################
# ONNX
#######################
# Require : image at "/content/drive/MyDrive/bus2.jpg"

In [23]:
!python -m pip install paddle2onnx -q
!python -m pip install onnxruntime==1.12.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.5 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR
import os
os.chdir('/content/PaddleOCR')
!pwd

In [ ]:
!wget -nc -P ./inference https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar
!cd ./inference && tar xf en_PP-OCRv3_det_infer.tar && cd ..

!wget -nc  -P ./inference https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_infer.tar
!cd ./inference && tar xf en_PP-OCRv3_rec_infer.tar && cd ..

!wget -nc  -P ./inference https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar
!cd ./inference && tar xf ch_ppocr_mobile_v2.0_cls_infer.tar && cd ..

In [ ]:
!paddle2onnx --model_dir ./inference/en_PP-OCRv3_det_infer \
--model_filename inference.pdmodel \
--params_filename inference.pdiparams \
--save_file ./inference/det_onnx/model.onnx \
--opset_version 10 \
--input_shape_dict="{'x':[-1,3,-1,-1]}" \
--enable_onnx_checker True

!paddle2onnx --model_dir ./inference/en_PP-OCRv3_rec_infer \
--model_filename inference.pdmodel \
--params_filename inference.pdiparams \
--save_file ./inference/rec_onnx/model.onnx \
--opset_version 10 \
--input_shape_dict="{'x':[-1,3,-1,-1]}" \
--enable_onnx_checker True

!paddle2onnx --model_dir ./inference/ch_ppocr_mobile_v2.0_cls_infer \
--model_filename inference.pdmodel \
--params_filename inference.pdiparams \
--save_file ./inference/cls_onnx/model.onnx \
--opset_version 10 \
--input_shape_dict="{'x':[-1,3,-1,-1]}" \
--enable_onnx_checker True

In [ ]:
!python infer/predict_system.py --use_gpu=True --use_onnx=True \
--det_model_dir=./inference/det_onnx/model.onnx  \
--rec_model_dir=./inference/rec_onnx/model.onnx  \
--cls_model_dir=./inference/cls_onnx/model.onnx  \
--image_dir='/content/drive/MyDrive/bus2.jpg' \
--rec_char_dict_path=ppocr/utils/dict/korean_dict.txt
